In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.optimize      import fsolve, curve_fit
from scipy.integrate     import odeint, quad
from scipy.interpolate   import interp1d
from scipy.misc          import derivative
import scipy.constants   as const
import sympy             as sp
sp.init_printing()
import glob
import time
import math
import alpaca_trade_api as tradeapi
from alpaca_trade_api import StreamConn
import threading
import time
import datetime
import logging
import argparse
import time
import pandas as pd
ALPACA_BASE_URL ='https://paper-api.alpaca.markets';
KEY = 'PK6Q22HTB5JPL4O35ZHM'
SCRKEY = 'sqv5r84hE7j1rqDVtUH5yC9qUEhJoND07DvuK3mG'

from lxml import html
import requests
import json
import argparse
from collections import OrderedDict

In [ ]:
def ski(symbol, large=15, medium=10, small=5, time_scale = 'day'):
    #EMA
    api = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
    Data = api.get_barset(symbol, time_scale, limit=1000).df
    LEMA = Data[symbol]['close'].rolling(window=large).mean()
    MEMA = Data[symbol]['close'].rolling(window=medium).mean()
    SEMA = Data[symbol]['close'].rolling(window=small).mean()
    
    #ski small
    #ski_s = (SEMA[-1]-MEMA[-1])/MEMA[-1]*100
    
    #ski large
    ski_l = (MEMA[-1]-LEMA[-1])/LEMA[-1]*100
    
    #delta ski
    Δski = ski_l
    return Δski

### EVALUATION

In [ ]:
def evaluation(stocks, Large=15, Medium=10, Small=5, Time_scale = 'day', accepted=10):
    #LISTING SKI VALUES
    List = []
    for i in range(len(stocks)):
        try: #symbol, large=15, medium=10, small=5, time_scale = 'day'
            List.append([stocks[i], ski(stocks[i], large=Large, medium=Medium, small=Small, time_scale = Time_scale)])
        #time.sleep()
        except:
            List.append([stocks[i], 0])
    
    #DELETING NULL VALUED MEMBERS
    values = []
    for row in List:
        values.append(row[-1])    
    for i in range(len(List)):
        if pd.notnull(values[i])== False:
            del List[i]
     
    #SORTING     
    value_list = []
    for rows in List:
        value_list.append(rows[-1])
    b = sorted(value_list)
    List_sorted = []
    for i in range(len(b)):
        INDEX = value_list.index(b[i])
        List_sorted.append((List[INDEX]))
        
    #LIST OF BEST STOCKS
    Best       = []
    for row in List_sorted:
        Best.append(row[0])
    return List_sorted[-accepted:], Best[-accepted:]

## The bounty jumper

#### PID BUYING COEFFICIENT

In [ ]:
def QUANTITY(CLOSE,L0 = 110000):
    api = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
    account = api.get_account()
    
    MONEY        = int(float(account.buying_power))-300000
    MONEY_TO_BUY = 0.005*MONEY
    N            = math.ceil(MONEY_TO_BUY/CLOSE)
    return N

In [ ]:
%%time
QUANTITY(10)

In [ ]:
def Crossover(symbol, large=20, medium=15, small=5):
    
    #try:
        api = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
        sb = api.get_barset(symbol, 'minute', limit = 3).df    
        update = []
        
        #CALCULATING THE TIME
        import time 
        TIME = time.localtime()
        minute = TIME.tm_min
        hr = TIME.tm_hour   
        if hr==7 and minute>28 or hr==13 and minute<40 or 13>hr>=8:
            t = 1
        else:
            t = 0
        
        #QUANTITY
        try:
            position = api.get_position(symbol)  #will this give me other useful data?
            quantity = position.qty
        except:
            quantity = 0
            update.append(',Market closed')
        #GETTING COHLV DATA
        close = sb[symbol]['close']
        open  = sb[symbol]['open']
        high  = sb[symbol]['high']
        low   = sb[symbol]['low']
        volume= sb[symbol]['volume']
    
        #EMA
        Data = api.get_barset(symbol, 'minute', limit=100).df
        LEMA = Data[symbol]['close'].rolling(window=large).mean()
        MEMA = Data[symbol]['close'].rolling(window=medium).mean()
        SEMA = Data[symbol]['close'].rolling(window=small).mean()
        
        #BUY
        if LEMA[-1]<MEMA[-1] and t==1 and close[1]>open[1]:
            api.submit_order(symbol, QUANTITY(close[1]), 'buy', 'market', 'day')  #consider making a limit order to prevent buying late into the game
            update.append(f'bought at {close[1]}')
        #SELL AT PLATEAU
        try:        
            if close[1]<open[1] and int(quantity)>0 and t==1:  # compare last two opens? #use a small EMA and copmare the 3 and last element, 2nd and last if on the 2nd
                api.submit_order(symbol, int(quantity),'sell', 'market', 'day')
                update.append(f'sold at {close[1]}')
        #SELL AT END OF DAY 
            if hr == 13 and minute>40 and int(quantity)>0:
                api.submit_order(symbol, int(quantity), 'sell', 'market', 'day')
                update.append(f'sold at {close[1]}')
        except:
            update.append(',Failure to sell')
        return print(symbol, ':', update)

In [ ]:
stocks = ('TSLA','ACB', 'STLA','F', 'AFRM','PKI','GE',
          'MSGM','PLUG','CGEM','SPCE','CCL','OGI',
          'HEXO','IDEX','TLRY','BNGO','RIOT', 'DRVN','TSLA','SHOP', 'API','NVAX', 'NTIC', 'COHU', 'UFS', 'FLGT', 
          'LPX')
A = evaluation(stocks)[1]
#use profit predicting code to add highest 5 out of 30 stocks based on highest yesterday
import asyncio
times = []
times.append(0)

j = 0   #async here and for the websocket   
while True:
    try:
        for i in range(len(A)):
            close = Crossover(A[i])
            print(close)
        times.append(time.localtime().tm_sec)
        j+=1
        print('time:',(times[j]-times[j-1])/len(stocks))
        time.sleep(1)
    except:
        print('retrying to call function')
#async to make the process faster, here and above
#make sure that I only buy at the beggining of the minute and not throughout.  
#use the order number example from alpaca examples to calculate the profit after each sell
#trading strategy: if ema is relatively linear and m>0 buy at the current EMA price (limit order ) and sell at predicted price in a short time


In [ ]:
#find out which stocks have the largest average magnitude over several days/ what is the most populated interval
#https://alpaca.markets/learn/algorithmic-trading-chart-pattern-python/
#go through all stock symbols, alpaca, label, and pick the best.  

### Analysis

In [ ]:
api = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
sb = api.get_barset('NVDA', 'minute', limit = 100).df   
high  = sb['NVDA']['high']
a = np.flip(high.values)


In [ ]:
fig=plt.figure(figsize=(10,8), dpi= 50, facecolor='w', edgecolor='k')
t = np.flip(high.index)
#t = (high.index)
D = a
plt.plot(t,a)
plt.xticks(rotation=45);

plt.show()